In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from catboost import CatBoostRegressor
from tqdm import tqdm

warnings.filterwarnings('ignore')

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
sample_submission = pd.read_csv("./data/sample_submission.csv")
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

categorial_variable = ["임대건물구분", "지역", "공급유형", "자격유형"]
continious_variable = ["총세대수", "전용면적", "전용면적별세대수", "공가수", "임대보증금", "임대료",
                       "단지내주차면수"]

# test_x_unique
# gbr: 72.57647194273466
# rfr: 19.73668358714044
# {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 16, 'n_estimators': 100} -62.07652539964124
# 144.17 143.65


C:\Users\mt_9\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\mt_9\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\mt_9\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\mt_9\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
def pre_processing(x, flag):
    x = x.fillna(0)
    x.loc[x['임대료'] == '-', ['임대료']] = 0
    x.loc[x['임대보증금'] == '-', ['임대보증금']] = 0
    x[['임대료', '임대보증금']] = x[['임대료', '임대보증금']].astype('int64')

    x['전용면적'] = x['전용면적'] // 5 * 5
    idx = x[x['전용면적'] > 100].index
    x.loc[idx, '전용면적'] = 100
    idx = x[x['전용면적'] < 15].index
    x.loc[idx, '전용면적'] = 15
    columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스', '공급유형', '임대건물구분', '자격유형']
    target = "등록차량수"
    area_columns = []
    for area in x['전용면적'].unique():
        area_columns.append(f'면적_{area}')

    new_x = pd.DataFrame()
    for i, code in tqdm(enumerate(x['단지코드'].unique())):
        temp = x[x['단지코드'] == code]
        temp.index = range(temp.shape[0])
        for col in columns:
            new_x.loc[i, col] = temp.loc[0, col]

        for col in area_columns:
            area = float(col.split('_')[-1])
            new_x.loc[i, col] = temp[temp['전용면적'] == area]['전용면적별세대수'].sum()

        if flag == True:
            new_x.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']

    new_x = pd.get_dummies(new_x, columns=categorial_variable)
    new_x = new_x[[c for c in new_x if c not in ['등록차량수']] + [c for c in ['등록차량수'] if c in new_x]]
    # scaler = RobustScaler()
    # x[continious_variable] = scaler.fit_transform(x[continious_variable])
    return new_x

differ_variables = ['공급유형_공공임대(5년)', '공급유형_공공임대(10년)', '자격유형_B', '자격유형_F',
                    '지역_서울특별시', '공급유형_공공분양', '공급유형_장기전세', '자격유형_D',
                    '면적_65.0']

if len(test[test['자격유형'].isnull() == True]) > 0:
    test.loc[test['자격유형'].isnull() == True, ['자격유형']] = ('A', 'C')

new_train = pre_processing(train, True)
new_test = pre_processing(test, False)


423it [00:07, 59.21it/s]
150it [00:02, 68.22it/s]


In [12]:
for c in differ_variables:
    new_test[c] = 0

x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:, -1]
x_test = new_test.iloc[:, 1:]

rfr = RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_leaf=1,
                            min_samples_split=4, random_state=46)
cbr = CatBoostRegressor(loss_function='MAE', random_state=75)
model = cbr
train_X, test_X, train_y, test_y = train_test_split(x_train, y_train, test_size=0.2, random_state=93)


In [14]:
model.fit(train_X, train_y)
pred = model.predict(test_X)
print(mean_absolute_error(test_y, pred))


0:	learn: 283.3526914	total: 1.71ms	remaining: 1.71s
1:	learn: 277.6529646	total: 3.58ms	remaining: 1.78s
2:	learn: 272.0426706	total: 5.33ms	remaining: 1.77s
3:	learn: 267.5274606	total: 7.14ms	remaining: 1.78s
4:	learn: 262.5153113	total: 8.86ms	remaining: 1.76s
5:	learn: 257.5534289	total: 10.5ms	remaining: 1.73s
6:	learn: 252.7952919	total: 12.2ms	remaining: 1.73s
7:	learn: 248.7754335	total: 13.3ms	remaining: 1.65s
8:	learn: 243.8356421	total: 15ms	remaining: 1.65s
9:	learn: 240.2048065	total: 16.5ms	remaining: 1.64s
10:	learn: 236.6636442	total: 18.4ms	remaining: 1.65s
11:	learn: 233.2824885	total: 20ms	remaining: 1.64s
12:	learn: 229.6517607	total: 21.5ms	remaining: 1.63s
13:	learn: 225.8029634	total: 23.2ms	remaining: 1.64s
14:	learn: 222.2032764	total: 25ms	remaining: 1.64s
15:	learn: 219.4028334	total: 26.7ms	remaining: 1.64s
16:	learn: 216.7286336	total: 28.3ms	remaining: 1.64s
17:	learn: 213.9010598	total: 29.9ms	remaining: 1.63s
18:	learn: 210.6126722	total: 31.5ms	remaini

202:	learn: 74.7141608	total: 319ms	remaining: 1.25s
203:	learn: 74.4582461	total: 321ms	remaining: 1.25s
204:	learn: 74.1130945	total: 323ms	remaining: 1.25s
205:	learn: 73.9578112	total: 324ms	remaining: 1.25s
206:	learn: 73.6797270	total: 326ms	remaining: 1.25s
207:	learn: 73.3397016	total: 328ms	remaining: 1.25s
208:	learn: 73.0010387	total: 330ms	remaining: 1.25s
209:	learn: 72.8407464	total: 332ms	remaining: 1.25s
210:	learn: 72.6791232	total: 334ms	remaining: 1.25s
211:	learn: 72.1757150	total: 335ms	remaining: 1.25s
212:	learn: 71.8243121	total: 337ms	remaining: 1.24s
213:	learn: 71.6861818	total: 339ms	remaining: 1.24s
214:	learn: 71.5213770	total: 341ms	remaining: 1.24s
215:	learn: 71.2979971	total: 342ms	remaining: 1.24s
216:	learn: 71.0953130	total: 344ms	remaining: 1.24s
217:	learn: 70.9427486	total: 345ms	remaining: 1.24s
218:	learn: 70.7365929	total: 347ms	remaining: 1.24s
219:	learn: 70.5833649	total: 348ms	remaining: 1.23s
220:	learn: 70.4310878	total: 350ms	remaining:

414:	learn: 47.0410067	total: 637ms	remaining: 898ms
415:	learn: 46.9965120	total: 641ms	remaining: 900ms
416:	learn: 46.8751340	total: 645ms	remaining: 902ms
417:	learn: 46.8133340	total: 649ms	remaining: 904ms
418:	learn: 46.7563035	total: 651ms	remaining: 902ms
419:	learn: 46.6528451	total: 652ms	remaining: 901ms
420:	learn: 46.6210222	total: 654ms	remaining: 899ms
421:	learn: 46.4538051	total: 656ms	remaining: 898ms
422:	learn: 46.4001531	total: 657ms	remaining: 897ms
423:	learn: 46.2865111	total: 659ms	remaining: 895ms
424:	learn: 46.2283116	total: 660ms	remaining: 894ms
425:	learn: 46.0961748	total: 662ms	remaining: 892ms
426:	learn: 46.0142388	total: 663ms	remaining: 890ms
427:	learn: 45.8828364	total: 665ms	remaining: 888ms
428:	learn: 45.8478571	total: 666ms	remaining: 887ms
429:	learn: 45.7488803	total: 668ms	remaining: 885ms
430:	learn: 45.6839150	total: 669ms	remaining: 883ms
431:	learn: 45.6553244	total: 671ms	remaining: 882ms
432:	learn: 45.5267604	total: 672ms	remaining:

621:	learn: 35.1826305	total: 954ms	remaining: 580ms
622:	learn: 35.1714160	total: 956ms	remaining: 578ms
623:	learn: 35.1079493	total: 957ms	remaining: 577ms
624:	learn: 35.0766881	total: 959ms	remaining: 575ms
625:	learn: 35.0203195	total: 961ms	remaining: 574ms
626:	learn: 34.9289223	total: 963ms	remaining: 573ms
627:	learn: 34.9219115	total: 964ms	remaining: 571ms
628:	learn: 34.8627405	total: 966ms	remaining: 570ms
629:	learn: 34.8227866	total: 967ms	remaining: 568ms
630:	learn: 34.8087156	total: 969ms	remaining: 566ms
631:	learn: 34.8000136	total: 970ms	remaining: 565ms
632:	learn: 34.7189664	total: 971ms	remaining: 563ms
633:	learn: 34.6878103	total: 973ms	remaining: 562ms
634:	learn: 34.6719176	total: 974ms	remaining: 560ms
635:	learn: 34.6336446	total: 976ms	remaining: 558ms
636:	learn: 34.6170840	total: 977ms	remaining: 557ms
637:	learn: 34.5998431	total: 979ms	remaining: 555ms
638:	learn: 34.5838142	total: 980ms	remaining: 554ms
639:	learn: 34.5720958	total: 982ms	remaining:

828:	learn: 29.0774536	total: 1.27s	remaining: 262ms
829:	learn: 29.0743324	total: 1.27s	remaining: 260ms
830:	learn: 29.0675060	total: 1.27s	remaining: 259ms
831:	learn: 29.0562988	total: 1.27s	remaining: 257ms
832:	learn: 29.0409751	total: 1.27s	remaining: 256ms
833:	learn: 29.0201591	total: 1.28s	remaining: 254ms
834:	learn: 28.9730157	total: 1.28s	remaining: 253ms
835:	learn: 28.9544424	total: 1.28s	remaining: 251ms
836:	learn: 28.9435019	total: 1.28s	remaining: 250ms
837:	learn: 28.8905961	total: 1.28s	remaining: 248ms
838:	learn: 28.8478278	total: 1.28s	remaining: 247ms
839:	learn: 28.8108213	total: 1.29s	remaining: 245ms
840:	learn: 28.8053919	total: 1.29s	remaining: 244ms
841:	learn: 28.8021527	total: 1.29s	remaining: 242ms
842:	learn: 28.7769444	total: 1.29s	remaining: 241ms
843:	learn: 28.7171724	total: 1.29s	remaining: 239ms
844:	learn: 28.7046123	total: 1.29s	remaining: 238ms
845:	learn: 28.6542342	total: 1.3s	remaining: 236ms
846:	learn: 28.6283459	total: 1.3s	remaining: 2

In [13]:
cv_score = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
print(np.mean(cv_score * -1))


136.19489651617266


In [11]:
params = {
        'n_estimators': [200],
        'max_depth': [15],
        'min_samples_leaf': [1],
        'min_samples_split': [4],
    }

cat_params = {
    'random_state':range(1,100)
}
grid = GridSearchCV(model, param_grid=cat_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid.fit(x_train, y_train)
pred = grid.predict(x_test)

print(grid.best_params_, grid.best_score_)
print(mean_absolute_error(x_test['단지내주차면수'],pred))

0:	learn: 283.4838998	total: 2.21ms	remaining: 2.2s
1:	learn: 279.1680300	total: 4.14ms	remaining: 2.06s
2:	learn: 273.9904908	total: 5.95ms	remaining: 1.98s
3:	learn: 269.5507408	total: 7.83ms	remaining: 1.95s
4:	learn: 264.7767351	total: 9.63ms	remaining: 1.92s
5:	learn: 260.3462801	total: 11.4ms	remaining: 1.89s
6:	learn: 255.8136531	total: 13.1ms	remaining: 1.86s
7:	learn: 251.3105353	total: 14.9ms	remaining: 1.85s
8:	learn: 247.3933304	total: 16.8ms	remaining: 1.84s
9:	learn: 243.1164277	total: 18.6ms	remaining: 1.84s
10:	learn: 239.1030658	total: 20.4ms	remaining: 1.83s
11:	learn: 234.5387823	total: 22ms	remaining: 1.81s
12:	learn: 231.2316044	total: 23.8ms	remaining: 1.81s
13:	learn: 227.6521476	total: 25.7ms	remaining: 1.81s
14:	learn: 223.4779998	total: 27.5ms	remaining: 1.81s
15:	learn: 220.2643857	total: 29.5ms	remaining: 1.81s
16:	learn: 217.0670794	total: 31.3ms	remaining: 1.81s
17:	learn: 213.4689956	total: 33.2ms	remaining: 1.81s
18:	learn: 210.0268616	total: 35ms	remain

156:	learn: 95.9609733	total: 310ms	remaining: 1.66s
157:	learn: 95.5386200	total: 312ms	remaining: 1.66s
158:	learn: 95.2807903	total: 314ms	remaining: 1.66s
159:	learn: 95.1279432	total: 316ms	remaining: 1.66s
160:	learn: 94.9473614	total: 318ms	remaining: 1.66s
161:	learn: 94.7440945	total: 320ms	remaining: 1.65s
162:	learn: 94.4538652	total: 322ms	remaining: 1.65s
163:	learn: 93.9349618	total: 323ms	remaining: 1.65s
164:	learn: 93.8050060	total: 324ms	remaining: 1.64s
165:	learn: 93.4246940	total: 328ms	remaining: 1.65s
166:	learn: 93.2065797	total: 331ms	remaining: 1.65s
167:	learn: 92.8510584	total: 333ms	remaining: 1.65s
168:	learn: 92.6834201	total: 336ms	remaining: 1.65s
169:	learn: 92.5123579	total: 338ms	remaining: 1.65s
170:	learn: 92.3196142	total: 341ms	remaining: 1.65s
171:	learn: 92.1663726	total: 343ms	remaining: 1.65s
172:	learn: 91.8204020	total: 345ms	remaining: 1.65s
173:	learn: 91.4019542	total: 347ms	remaining: 1.65s
174:	learn: 91.0870738	total: 349ms	remaining:

394:	learn: 57.6637772	total: 781ms	remaining: 1.2s
395:	learn: 57.6089481	total: 783ms	remaining: 1.19s
396:	learn: 57.5193539	total: 786ms	remaining: 1.19s
397:	learn: 57.4708819	total: 788ms	remaining: 1.19s
398:	learn: 57.4152810	total: 790ms	remaining: 1.19s
399:	learn: 57.3398345	total: 792ms	remaining: 1.19s
400:	learn: 57.2289347	total: 793ms	remaining: 1.19s
401:	learn: 57.0808377	total: 795ms	remaining: 1.18s
402:	learn: 57.0605463	total: 797ms	remaining: 1.18s
403:	learn: 56.9871333	total: 799ms	remaining: 1.18s
404:	learn: 56.9359185	total: 801ms	remaining: 1.18s
405:	learn: 56.8550168	total: 803ms	remaining: 1.17s
406:	learn: 56.7851776	total: 804ms	remaining: 1.17s
407:	learn: 56.7379060	total: 806ms	remaining: 1.17s
408:	learn: 56.5959525	total: 808ms	remaining: 1.17s
409:	learn: 56.5279626	total: 810ms	remaining: 1.17s
410:	learn: 56.3669374	total: 813ms	remaining: 1.16s
411:	learn: 56.3177439	total: 814ms	remaining: 1.16s
412:	learn: 56.2171584	total: 816ms	remaining: 

568:	learn: 46.0423021	total: 1.1s	remaining: 835ms
569:	learn: 45.9104905	total: 1.1s	remaining: 833ms
570:	learn: 45.8635836	total: 1.1s	remaining: 831ms
571:	learn: 45.8532019	total: 1.11s	remaining: 829ms
572:	learn: 45.7874844	total: 1.11s	remaining: 827ms
573:	learn: 45.7484185	total: 1.11s	remaining: 825ms
574:	learn: 45.7081866	total: 1.11s	remaining: 823ms
575:	learn: 45.5874047	total: 1.11s	remaining: 821ms
576:	learn: 45.5266063	total: 1.12s	remaining: 819ms
577:	learn: 45.4617421	total: 1.12s	remaining: 816ms
578:	learn: 45.3788331	total: 1.12s	remaining: 814ms
579:	learn: 45.3015668	total: 1.12s	remaining: 812ms
580:	learn: 45.2630672	total: 1.12s	remaining: 810ms
581:	learn: 45.2350242	total: 1.12s	remaining: 807ms
582:	learn: 45.2047195	total: 1.13s	remaining: 805ms
583:	learn: 45.1393663	total: 1.13s	remaining: 803ms
584:	learn: 45.0980808	total: 1.13s	remaining: 801ms
585:	learn: 45.0240417	total: 1.13s	remaining: 798ms
586:	learn: 44.9799635	total: 1.13s	remaining: 79

767:	learn: 36.6742292	total: 1.42s	remaining: 429ms
768:	learn: 36.6649445	total: 1.42s	remaining: 427ms
769:	learn: 36.6443227	total: 1.42s	remaining: 425ms
770:	learn: 36.6090597	total: 1.43s	remaining: 423ms
771:	learn: 36.5930865	total: 1.43s	remaining: 422ms
772:	learn: 36.5853200	total: 1.43s	remaining: 420ms
773:	learn: 36.5492339	total: 1.43s	remaining: 418ms
774:	learn: 36.5290835	total: 1.43s	remaining: 416ms
775:	learn: 36.4880770	total: 1.43s	remaining: 414ms
776:	learn: 36.4741018	total: 1.44s	remaining: 412ms
777:	learn: 36.4669194	total: 1.44s	remaining: 410ms
778:	learn: 36.3656004	total: 1.44s	remaining: 408ms
779:	learn: 36.3584508	total: 1.44s	remaining: 406ms
780:	learn: 36.3485313	total: 1.44s	remaining: 404ms
781:	learn: 36.3231516	total: 1.44s	remaining: 402ms
782:	learn: 36.2252385	total: 1.45s	remaining: 400ms
783:	learn: 36.1988274	total: 1.45s	remaining: 399ms
784:	learn: 36.1517298	total: 1.45s	remaining: 397ms
785:	learn: 36.0629366	total: 1.45s	remaining:

969:	learn: 30.9193866	total: 1.74s	remaining: 53.7ms
970:	learn: 30.8873763	total: 1.74s	remaining: 51.9ms
971:	learn: 30.8680647	total: 1.74s	remaining: 50.1ms
972:	learn: 30.8412912	total: 1.74s	remaining: 48.4ms
973:	learn: 30.8203805	total: 1.74s	remaining: 46.6ms
974:	learn: 30.7824146	total: 1.75s	remaining: 44.8ms
975:	learn: 30.7771998	total: 1.75s	remaining: 43ms
976:	learn: 30.7544103	total: 1.75s	remaining: 41.2ms
977:	learn: 30.7395780	total: 1.75s	remaining: 39.4ms
978:	learn: 30.7272090	total: 1.75s	remaining: 37.6ms
979:	learn: 30.6971316	total: 1.75s	remaining: 35.8ms
980:	learn: 30.6813966	total: 1.75s	remaining: 34ms
981:	learn: 30.6773518	total: 1.76s	remaining: 32.2ms
982:	learn: 30.6625070	total: 1.76s	remaining: 30.4ms
983:	learn: 30.6331505	total: 1.76s	remaining: 28.6ms
984:	learn: 30.6016968	total: 1.76s	remaining: 26.8ms
985:	learn: 30.5929762	total: 1.76s	remaining: 25ms
986:	learn: 30.5811907	total: 1.76s	remaining: 23.2ms
987:	learn: 30.5571661	total: 1.77

In [10]:
model.fit(x_train, y_trai
df = pd.DataFrame()
df['column'] = x_train.columns
df['coef'] = model.feature_importances_
print(df.sort_values('coef', ascending=False).head(10))

SyntaxError: invalid syntax (<ipython-input-10-63aef8b1908f>, line 2)

In [15]:
model.fit(x_train, y_train)
pred = model.predict(x_test)
sample_submission['num'] = pred
sample_submission.to_csv('./result/result6_3.csv', index=False)

0:	learn: 283.4838998	total: 1.96ms	remaining: 1.96s
1:	learn: 279.1680300	total: 3.64ms	remaining: 1.82s
2:	learn: 273.9904908	total: 5.32ms	remaining: 1.77s
3:	learn: 269.5507408	total: 6.98ms	remaining: 1.74s
4:	learn: 264.7767351	total: 8.67ms	remaining: 1.73s
5:	learn: 260.3462801	total: 10.5ms	remaining: 1.74s
6:	learn: 255.8136531	total: 12.9ms	remaining: 1.84s
7:	learn: 251.3105353	total: 14.7ms	remaining: 1.83s
8:	learn: 247.3933304	total: 16.5ms	remaining: 1.82s
9:	learn: 243.1164277	total: 18.2ms	remaining: 1.8s
10:	learn: 239.1030658	total: 19.9ms	remaining: 1.79s
11:	learn: 234.5387823	total: 21.5ms	remaining: 1.77s
12:	learn: 231.2316044	total: 23.2ms	remaining: 1.76s
13:	learn: 227.6521476	total: 25.4ms	remaining: 1.79s
14:	learn: 223.4779998	total: 27.4ms	remaining: 1.8s
15:	learn: 220.2643857	total: 29.1ms	remaining: 1.79s
16:	learn: 217.0670794	total: 30.7ms	remaining: 1.77s
17:	learn: 213.4689956	total: 32.3ms	remaining: 1.76s
18:	learn: 210.0268616	total: 33.9ms	rem

196:	learn: 86.5221405	total: 315ms	remaining: 1.28s
197:	learn: 86.3083380	total: 317ms	remaining: 1.28s
198:	learn: 85.9360258	total: 319ms	remaining: 1.28s
199:	learn: 85.7129916	total: 321ms	remaining: 1.28s
200:	learn: 85.4170780	total: 323ms	remaining: 1.28s
201:	learn: 85.3047466	total: 324ms	remaining: 1.28s
202:	learn: 85.2126941	total: 326ms	remaining: 1.28s
203:	learn: 85.0500866	total: 328ms	remaining: 1.28s
204:	learn: 84.6730697	total: 330ms	remaining: 1.28s
205:	learn: 84.4453408	total: 332ms	remaining: 1.28s
206:	learn: 84.1565982	total: 334ms	remaining: 1.28s
207:	learn: 83.9902365	total: 336ms	remaining: 1.28s
208:	learn: 83.7795706	total: 338ms	remaining: 1.28s
209:	learn: 83.6296543	total: 340ms	remaining: 1.28s
210:	learn: 83.3988892	total: 341ms	remaining: 1.28s
211:	learn: 83.2488283	total: 343ms	remaining: 1.27s
212:	learn: 83.0158890	total: 345ms	remaining: 1.27s
213:	learn: 82.9112133	total: 347ms	remaining: 1.27s
214:	learn: 82.5443772	total: 348ms	remaining:

389:	learn: 58.0270540	total: 631ms	remaining: 987ms
390:	learn: 57.9534402	total: 633ms	remaining: 985ms
391:	learn: 57.8454450	total: 634ms	remaining: 984ms
392:	learn: 57.8322472	total: 636ms	remaining: 982ms
393:	learn: 57.7205242	total: 638ms	remaining: 981ms
394:	learn: 57.6637772	total: 640ms	remaining: 980ms
395:	learn: 57.6089481	total: 642ms	remaining: 980ms
396:	learn: 57.5193539	total: 644ms	remaining: 978ms
397:	learn: 57.4708819	total: 645ms	remaining: 976ms
398:	learn: 57.4152810	total: 647ms	remaining: 975ms
399:	learn: 57.3398345	total: 649ms	remaining: 973ms
400:	learn: 57.2289347	total: 650ms	remaining: 972ms
401:	learn: 57.0808377	total: 652ms	remaining: 970ms
402:	learn: 57.0605463	total: 654ms	remaining: 968ms
403:	learn: 56.9871333	total: 655ms	remaining: 967ms
404:	learn: 56.9359185	total: 657ms	remaining: 965ms
405:	learn: 56.8550168	total: 658ms	remaining: 963ms
406:	learn: 56.7851776	total: 660ms	remaining: 961ms
407:	learn: 56.7379060	total: 661ms	remaining:

561:	learn: 46.4872524	total: 948ms	remaining: 739ms
562:	learn: 46.4682022	total: 949ms	remaining: 737ms
563:	learn: 46.3988842	total: 951ms	remaining: 735ms
564:	learn: 46.3597488	total: 953ms	remaining: 733ms
565:	learn: 46.3215850	total: 954ms	remaining: 732ms
566:	learn: 46.2183356	total: 956ms	remaining: 730ms
567:	learn: 46.1384652	total: 957ms	remaining: 728ms
568:	learn: 46.0423021	total: 958ms	remaining: 726ms
569:	learn: 45.9104905	total: 961ms	remaining: 725ms
570:	learn: 45.8635836	total: 963ms	remaining: 723ms
571:	learn: 45.8532019	total: 965ms	remaining: 722ms
572:	learn: 45.7874844	total: 967ms	remaining: 721ms
573:	learn: 45.7484185	total: 969ms	remaining: 719ms
574:	learn: 45.7081866	total: 970ms	remaining: 717ms
575:	learn: 45.5874047	total: 972ms	remaining: 715ms
576:	learn: 45.5266063	total: 974ms	remaining: 714ms
577:	learn: 45.4617421	total: 976ms	remaining: 712ms
578:	learn: 45.3788331	total: 977ms	remaining: 711ms
579:	learn: 45.3015668	total: 979ms	remaining:

742:	learn: 37.5650793	total: 1.26s	remaining: 437ms
743:	learn: 37.5311527	total: 1.26s	remaining: 435ms
744:	learn: 37.5043873	total: 1.26s	remaining: 433ms
745:	learn: 37.4710872	total: 1.27s	remaining: 431ms
746:	learn: 37.4317715	total: 1.27s	remaining: 430ms
747:	learn: 37.3867141	total: 1.27s	remaining: 428ms
748:	learn: 37.3795020	total: 1.27s	remaining: 427ms
749:	learn: 37.3532342	total: 1.27s	remaining: 425ms
750:	learn: 37.3001832	total: 1.28s	remaining: 423ms
751:	learn: 37.2729406	total: 1.28s	remaining: 422ms
752:	learn: 37.2067976	total: 1.28s	remaining: 420ms
753:	learn: 37.1600353	total: 1.28s	remaining: 418ms
754:	learn: 37.1491222	total: 1.28s	remaining: 417ms
755:	learn: 37.1121333	total: 1.28s	remaining: 415ms
756:	learn: 37.0979942	total: 1.29s	remaining: 413ms
757:	learn: 37.0642735	total: 1.29s	remaining: 411ms
758:	learn: 37.0541321	total: 1.29s	remaining: 410ms
759:	learn: 37.0377091	total: 1.29s	remaining: 408ms
760:	learn: 37.0118937	total: 1.29s	remaining:

934:	learn: 31.7177644	total: 1.58s	remaining: 110ms
935:	learn: 31.7071737	total: 1.58s	remaining: 108ms
936:	learn: 31.6973112	total: 1.58s	remaining: 106ms
937:	learn: 31.6808241	total: 1.58s	remaining: 105ms
938:	learn: 31.6786852	total: 1.58s	remaining: 103ms
939:	learn: 31.6419501	total: 1.58s	remaining: 101ms
940:	learn: 31.5769252	total: 1.59s	remaining: 99.5ms
941:	learn: 31.5407420	total: 1.59s	remaining: 97.8ms
942:	learn: 31.5001696	total: 1.59s	remaining: 96.1ms
943:	learn: 31.4724930	total: 1.59s	remaining: 94.5ms
944:	learn: 31.4413708	total: 1.59s	remaining: 92.8ms
945:	learn: 31.4209318	total: 1.6s	remaining: 91.1ms
946:	learn: 31.3865985	total: 1.6s	remaining: 89.4ms
947:	learn: 31.3667427	total: 1.6s	remaining: 87.8ms
948:	learn: 31.3535096	total: 1.6s	remaining: 86.1ms
949:	learn: 31.3378352	total: 1.6s	remaining: 84.4ms
950:	learn: 31.3340123	total: 1.6s	remaining: 82.7ms
951:	learn: 31.3150421	total: 1.61s	remaining: 81ms
952:	learn: 31.2703948	total: 1.61s	remain

In [16]:
best_score = pd.read_csv('./result/result4_2.csv')['num']
current_score = pd.read_csv('./result/result6_3.csv')['num']
mean_absolute_error(best_score, current_score)

50.48914512148051